In [ ]:
!nvidia-smi

Sat Jul 10 16:59:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-cp37-none-any.whl size=16443 sha256=750c76b9d9dd5d0a03aa7fdd04648f68a267685b68bc710eadc730b942b5d627
  Stored in directory: /root/.cache/pip/wheels/84/27/aa/c46d23c4e8cc72d41283862b1437e0b3ad318417e8ed7d5921
Successfully built efficientnet-pytorch


In [ ]:
!unzip '/content/drive/MyDrive/FewShot.zip'

Streaming output truncated to the last 5000 lines.
  inflating: FewShot/fss1000/refrigerator/label/10.png  
  inflating: FewShot/fss1000/refrigerator/label/2.png  
  inflating: FewShot/fss1000/refrigerator/label/3.png  
  inflating: FewShot/fss1000/refrigerator/label/4.png  
  inflating: FewShot/fss1000/refrigerator/label/5.png  
  inflating: FewShot/fss1000/refrigerator/label/6.png  
  inflating: FewShot/fss1000/refrigerator/label/7.png  
  inflating: FewShot/fss1000/refrigerator/label/8.png  
  inflating: FewShot/fss1000/refrigerator/label/9.png  
   creating: FewShot/fss1000/relay_stick/
   creating: FewShot/fss1000/relay_stick/image/
  inflating: FewShot/fss1000/relay_stick/image/1.jpg  
  inflating: FewShot/fss1000/relay_stick/image/10.jpg  
  inflating: FewShot/fss1000/relay_stick/image/2.jpg  
  inflating: FewShot/fss1000/relay_stick/image/3.jpg  
  inflating: FewShot/fss1000/relay_stick/image/4.jpg  
  inflating: FewShot/fss1000/relay_stick/image/5.jpg  
  inflating: FewShot/fs

In [ ]:
!ls

drive  FewShot	sample_data


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
import torchvision.models as models
import numpy as np
import os
import subprocess
import math
import argparse
import random
import cv2
import sys
sys.argv=['']
del sys

parser = argparse.ArgumentParser(description="One Shot Visual Recognition")
parser.add_argument("-w","--class_num",type = int, default = 1)
parser.add_argument("-s","--sample_num_per_class",type = int, default = 5)
parser.add_argument("-b","--batch_num_per_class",type = int, default = 5)
parser.add_argument("-e","--episode",type = int, default= 500000)
parser.add_argument("-t","--test_episode", type = int, default = 1000)
parser.add_argument("-l","--learning_rate", type = float, default = 0.001)
parser.add_argument("-rf","--TrainResultPath",type=str,default='result_newvgg_1shot')
parser.add_argument("-rff","--ResultSaveFreq",type=int,default=10000)
parser.add_argument("-msp","--ModelSavePath",type=str,default='models_newvgg_1shot')
parser.add_argument("-msf","--ModelSaveFreq",type=int,default=10000)
parser.add_argument("-g","--gpu",type=int, default=0)
parser.add_argument("-d","--display_query_num",type=int,default=5)
parser.add_argument("-modelf","--encoder_model",type=str,default='')
parser.add_argument("-modeld","--decoder_model",type=str,default='')
parser.add_argument("-modelh","--head_model",type=str,default='')
parser.add_argument("-start","--start_episode",type = int, default= 0)
parser.add_argument("-fi","--finetune",type=bool,default=True)



args = parser.parse_args()



## Hyper Parameters

CLASS_NUM = args.class_num
SAMPLE_NUM_PER_CLASS = args.sample_num_per_class
BATCH_NUM_PER_CLASS = args.batch_num_per_class
EPISODE = args.episode
TEST_EPISODE = args.test_episode
LEARNING_RATE = args.learning_rate
GPU = args.gpu
DISPLAY_QUERY = args.display_query_num
ENCODER_MODEL = args.encoder_model
DECODER_MODEL = args.decoder_model
HEAD_MODEL = args.head_model

In [ ]:
import torch
import torch.nn as nn


def patch_first_conv(model, in_channels):
    """Change first convolution layer input channels.
    In case:
        in_channels == 1 or in_channels == 2 -> reuse original weights
        in_channels > 3 -> make random kaiming normal initialization
    """

    # get first conv
    for module in model.modules():
        if isinstance(module, nn.Conv2d):
            break

    # change input channels for first conv
    module.in_channels = in_channels
    weight = module.weight.detach()
    reset = False

    if in_channels == 1:
        weight = weight.sum(1, keepdim=True)
    elif in_channels == 2:
        weight = weight[:, :2] * (3.0 / 2.0)
    else:
        reset = True
        weight = torch.Tensor(
            module.out_channels,
            module.in_channels // module.groups,
            *module.kernel_size
        )

    module.weight = nn.parameter.Parameter(weight)
    if reset:
        module.reset_parameters()


def replace_strides_with_dilation(module, dilation_rate):
    """Patch Conv2d modules replacing strides with dilation"""
    for mod in module.modules():
        if isinstance(mod, nn.Conv2d):
            mod.stride = (1, 1)
            mod.dilation = (dilation_rate, dilation_rate)
            kh, kw = mod.kernel_size
            mod.padding = ((kh // 2) * dilation_rate, (kh // 2) * dilation_rate)

            # Kostyl for EfficientNet
            if hasattr(mod, "static_padding"):
                mod.static_padding = nn.Identity()


In [ ]:
import torch
import torch.nn as nn
from typing import List
from collections import OrderedDict



class EncoderMixin:
    """Add encoder functionality such as:
        - output channels specification of feature tensors (produced by encoder)
        - patching first convolution for arbitrary input channels
    """

    @property
    def out_channels(self):
        """Return channels dimensions for each tensor of forward output of encoder"""
        return self._out_channels[: self._depth + 1]

    def set_in_channels(self, in_channels):
        """Change first convolution channels"""
        if in_channels == 3:
            return

        self._in_channels = in_channels
        if self._out_channels[0] == 3:
            self._out_channels = tuple([in_channels] + list(self._out_channels)[1:])

        patch_first_conv(model=self, in_channels=in_channels)

    def get_stages(self):
        """Method should be overridden in encoder"""
        raise NotImplementedError

    def make_dilated(self, stage_list, dilation_list):
        stages = self.get_stages()
        for stage_indx, dilation_rate in zip(stage_list, dilation_list):
            replace_strides_with_dilation(
                module=stages[stage_indx],
                dilation_rate=dilation_rate,
            )


# Concurrent Channel and Spatial Excitation


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class ChannelSELayer(nn.Module):
    """
    Re-implementation of Squeeze-and-Excitation (SE) block described in:
        *Hu et al., Squeeze-and-Excitation Networks, arXiv:1709.01507*
    """

    def __init__(self, num_channels, reduction_ratio=2):
        """
        :param num_channels: No of input channels
        :param reduction_ratio: By how much should the num_channels should be reduced
        """
        super(ChannelSELayer, self).__init__()
        num_channels_reduced = num_channels // reduction_ratio
        self.reduction_ratio = reduction_ratio
        self.fc1 = nn.Linear(num_channels, num_channels_reduced, bias=True)
        self.fc2 = nn.Linear(num_channels_reduced, num_channels, bias=True)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_tensor):
        """
        :param input_tensor: X, shape = (batch_size, num_channels, H, W)
        :return: output tensor
        """
        batch_size, num_channels, H, W = input_tensor.size()
        # Average along each channel
        squeeze_tensor = input_tensor.view(batch_size, num_channels, -1).mean(dim=2)

        # channel excitation
        fc_out_1 = self.relu(self.fc1(squeeze_tensor))
        fc_out_2 = self.sigmoid(self.fc2(fc_out_1))

        a, b = squeeze_tensor.size()
        output_tensor = torch.mul(input_tensor, fc_out_2.view(a, b, 1, 1))
        return output_tensor


class SpatialSELayer(nn.Module):
    """
    Re-implementation of SE block -- squeezing spatially and exciting channel-wise described in:
        *Roy et al., Concurrent Spatial and Channel Squeeze & Excitation in Fully Convolutional Networks, MICCAI 2018*
    """

    def __init__(self, num_channels):
        """
        :param num_channels: No of input channels
        """
        super(SpatialSELayer, self).__init__()
        self.conv = nn.Conv2d(num_channels, 1, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_tensor, weights=None):
        """
        :param weights: weights for few shot learning
        :param input_tensor: X, shape = (batch_size, num_channels, H, W)
        :return: output_tensor
        """
        # spatial squeeze
        batch_size, channel, a, b = input_tensor.size()

        if weights is not None:
            weights = torch.mean(weights, dim=0)
            weights = weights.view(1, channel, 1, 1)
            out = F.conv2d(input_tensor, weights)
        else:
            out = self.conv(input_tensor)
        squeeze_tensor = self.sigmoid(out)

        # spatial excitation
        # print(input_tensor.size(), squeeze_tensor.size())
        squeeze_tensor = squeeze_tensor.view(batch_size, 1, a, b)
        output_tensor = torch.mul(input_tensor, squeeze_tensor)
        #output_tensor = torch.mul(input_tensor, squeeze_tensor)
        return output_tensor


class ChannelSpatialSELayer(nn.Module):
    """
    Re-implementation of concurrent spatial and channel squeeze & excitation:
        *Roy et al., Concurrent Spatial and Channel Squeeze & Excitation in Fully Convolutional Networks, MICCAI 2018, arXiv:1803.02579*
    """

    def __init__(self, num_channels, reduction_ratio=2):
        """
        :param num_channels: No of input channels
        :param reduction_ratio: By how much should the num_channels should be reduced
        """
        super(ChannelSpatialSELayer, self).__init__()
        self.cSE = ChannelSELayer(num_channels, reduction_ratio).cuda(GPU)
        self.sSE = SpatialSELayer(num_channels).cuda(GPU)

    def forward(self, input_tensor):
        """
        :param input_tensor: X, shape = (batch_size, num_channels, H, W)
        :return: output_tensor
        """
        output_tensor = torch.max(self.cSE(input_tensor), self.sSE(input_tensor))
        return output_tensor


In [ ]:
import torch.nn as nn
from efficientnet_pytorch import EfficientNet, model
from efficientnet_pytorch.utils import url_map, url_map_advprop, get_model_params



class EfficientNetEncoder(EfficientNet, EncoderMixin):
    def __init__(self, stage_idxs, out_channels, model_name, depth=5):

        blocks_args, global_params = get_model_params(model_name, override_params=None)
        super().__init__(blocks_args, global_params)

        self._stage_idxs = stage_idxs
        self._out_channels = out_channels
        self._depth = depth
        self._in_channels = 3
        #self.scse = ChannelSpatialSELayer(self._in_channels)
       

        del self._fc

    def get_stages(self):
        return [
            nn.Identity(),
            nn.Sequential(self._conv_stem, self._bn0, self._swish),
            self._blocks[:self._stage_idxs[0]],
            self._blocks[self._stage_idxs[0]:self._stage_idxs[1]],
            self._blocks[self._stage_idxs[1]:self._stage_idxs[2]],
            self._blocks[self._stage_idxs[2]:],
        ]

    def forward(self, x):
        stages = self.get_stages()
        #print(stages)

        block_number = 0.
        drop_connect_rate = self._global_params.drop_connect_rate
        #print(drop_connect_rate)

        features = []
        for i in range(self._depth + 1):

            # Identity and Sequential stages
            if i < 2:
                x = stages[i](x)

            # Block stages need drop_connect rate
            else:
                for module in stages[i]:
                    drop_connect = drop_connect_rate * block_number / len(self._blocks)
                    block_number += 1.
                    x = module(x, drop_connect)

            features.append(x)
            in_ch = x.shape[1]
            #print(in_ch)
            scse = ChannelSpatialSELayer(in_ch)
            x = scse(x)
            #print(x.shape)
            
          
        return x,features

def _get_pretrained_settings(encoder):
    pretrained_settings = {
        "imagenet": {
            "mean": [0.485, 0.456, 0.406],
            "std": [0.229, 0.224, 0.225],
            "url": url_map[encoder],
            "input_space": "RGB",
            "input_range": [0, 1],
        },
    }
    return pretrained_settings

efficient_net_encoders = {
    "efficientnet-b4": {
        "encoder": EfficientNetEncoder,
        "pretrained_settings": _get_pretrained_settings("efficientnet-b4"),
        "params": {
            "out_channels": (3, 48, 32, 56, 160, 448),
            "stage_idxs": (6, 10, 22, 32),
            "model_name": "efficientnet-b4",
        },
    },
}



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F



try:
    from inplace_abn import InPlaceABN
except ImportError:
    InPlaceABN = None

class Conv2dReLU(nn.Sequential):
    def __init__(
            self,
            in_channels,
            out_channels,
            kernel_size,
            padding=0,
            stride=1,
            use_batchnorm=True,
    ):

        if use_batchnorm == "inplace" and InPlaceABN is None:
            raise RuntimeError(
                "In order to use `use_batchnorm='inplace'` inplace_abn package must be installed. "
                + "To install see: https://github.com/mapillary/inplace_abn"
            )

        conv = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            stride=stride,
            padding=padding,
            bias=not (use_batchnorm),
        )
        relu = nn.ReLU(inplace=True)

        if use_batchnorm == "inplace":
            bn = InPlaceABN(out_channels, activation="leaky_relu", activation_param=0.0)
            relu = nn.Identity()

        elif use_batchnorm and use_batchnorm != "inplace":
            bn = nn.BatchNorm2d(out_channels)

        else:
            bn = nn.Identity()

        super(Conv2dReLU, self).__init__(conv, bn, relu)


class Conv2dReLU(nn.Sequential):
    def __init__(
            self,
            in_channels,
            out_channels,
            kernel_size,
            padding=0,
            stride=1,
            use_batchnorm=True,
    ):


        conv = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            stride=stride,
            padding=padding,
            bias=not (use_batchnorm),
        )
        relu = nn.ReLU(inplace=True)

        if use_batchnorm and use_batchnorm != "inplace":
            bn = nn.BatchNorm2d(out_channels)

        else:
            bn = nn.Identity()

        super(Conv2dReLU, self).__init__(conv, bn, relu)



class DecoderBlock(nn.Module):
    def __init__(
            self,
            in_channels,
            skip_channels,
            out_channels,
            use_batchnorm=True,
            attention_type='scse',

    ):
        super().__init__()


        self.conv1 = Conv2dReLU(
            in_channels + skip_channels,
            out_channels,
            kernel_size=3,
            padding=1,
            use_batchnorm=use_batchnorm,
        )
        self.attention1 = ChannelSpatialSELayer(num_channels=in_channels + skip_channels)
  
        self.conv2 = Conv2dReLU(
            out_channels,
            out_channels,
            kernel_size=3,
            padding=1,
            use_batchnorm=use_batchnorm,
        )

        self.attention2 = ChannelSpatialSELayer(num_channels=out_channels)


    def forward(self, x, skip=None):

        x = F.interpolate(x, scale_factor=2, mode="nearest")
       
        
        if skip is not None:
           
            x = torch.cat([x, skip], dim=1)
            
            
            x = self.attention1(x)
       
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.attention2(x)
        
        return x

class UnetDecoder(nn.Module):
    def __init__( self):
        super().__init__()

       

        in_channels = [896, 256, 128, 64, 32]
        skip_channels = [160, 56, 32, 48, 0]
        out_channels = [256, 128, 64, 32, 16]


      
        blocks = [
            DecoderBlock(in_ch, skip_ch, out_ch)
            for in_ch, skip_ch, out_ch in zip(in_channels, skip_channels, out_channels)
        ]
        self.blocks = nn.ModuleList(blocks)

    def forward(self,x,features):

       
        features = features[1:]                 # remove first skip with same spatial resolution, i.e, channel feature 768
       
        features = features[::-1]               # reverse channels to start from head of encoder  
     

        
        skips = features[1:]                    


       

        
        for i, decoder_block in enumerate(self.blocks):
            skip = skips[i] if i < len(skips) else None
            x = decoder_block(x, skip)
           

        return x



In [ ]:
model = UnetDecoder()
print(model)

UnetDecoder(
  (blocks): ModuleList(
    (0): DecoderBlock(
      (conv1): Conv2dReLU(
        (0): Conv2d(1056, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (attention1): ChannelSpatialSELayer(
        (cSE): ChannelSELayer(
          (fc1): Linear(in_features=1056, out_features=528, bias=True)
          (fc2): Linear(in_features=528, out_features=1056, bias=True)
          (relu): ReLU()
          (sigmoid): Sigmoid()
        )
        (sSE): SpatialSELayer(
          (conv): Conv2d(1056, 1, kernel_size=(1, 1), stride=(1, 1))
          (sigmoid): Sigmoid()
        )
      )
      (conv2): Conv2dReLU(
        (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )


In [ ]:
import torch.utils.model_zoo as model_zoo

encoders = {}
encoders.update(efficient_net_encoders)

def get_encoder(name, in_channels=3, depth=5, weights=None):

    if weights == None:
      print("Training from Scratch !!")

    try:
        Encoder = encoders[name]["encoder"]
       
    except KeyError:
        raise KeyError("Wrong encoder name `{}`, supported encoders: {}".format(name, list(encoders.keys())))



    params = encoders[name]["params"]
   
    
    params.update(depth=depth)
    
   
    encoder = Encoder(**params)

    if weights is not None:
        try:
            settings = encoders[name]["pretrained_settings"][weights]
        except KeyError:
            raise KeyError("Wrong pretrained weights `{}` for encoder `{}`. Available options are: {}".format(
                weights, name, list(encoders[name]["pretrained_settings"].keys()),
            ))
        encoder.load_state_dict(model_zoo.load_url(settings["url"]))

    encoder.set_in_channels(in_channels)
    

    return encoder

In [ ]:
class SegmentationHead(nn.Sequential):

    def __init__(self, in_channels, out_channels, kernel_size=3, activation='sigmoid', upsampling=1):
        conv2d = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=kernel_size // 2)
        upsampling = nn.UpsamplingBilinear2d(scale_factor=upsampling) if upsampling > 1 else nn.Identity()
        activation = nn.Sigmoid()
        super().__init__(conv2d, upsampling, activation)



In [ ]:
def initialize_decoder(module):
    for m in module.modules():

        if isinstance(m, nn.Conv2d):
            nn.init.kaiming_uniform_(m.weight, mode="fan_in", nonlinearity="relu")
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)

        elif isinstance(m, nn.BatchNorm2d):
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0)

        elif isinstance(m, nn.Linear):
            nn.init.xavier_uniform_(m.weight)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)


def initialize_head(module):
    for m in module.modules():
        if isinstance(m, (nn.Linear, nn.Conv2d)):
            nn.init.xavier_uniform_(m.weight)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)

In [ ]:
classes_name = os.listdir('FewShot/fss1000/')
classes = list(range(0,len(classes_name)))

chosen_classes = random.sample(classes, CLASS_NUM)

def get_oneshot_batch():
  
        # print ('class %s is chosen' % i)
      classes_name = os.listdir('FewShot/fss1000/')
      classes = list(range(0,len(classes_name)))
      

      chosen_classes = random.sample(classes, CLASS_NUM)
      #print(chosen_classes)
      support_images = np.zeros((CLASS_NUM*SAMPLE_NUM_PER_CLASS,3,224,224), dtype=np.float32)
      support_labels = np.zeros((CLASS_NUM*SAMPLE_NUM_PER_CLASS,CLASS_NUM,224,224), dtype=np.float32)
      query_images = np.zeros((CLASS_NUM*BATCH_NUM_PER_CLASS,3,224,224), dtype=np.float32)
      query_labels = np.zeros((CLASS_NUM*BATCH_NUM_PER_CLASS,CLASS_NUM,224,224), dtype=np.float32)
      zeros = np.zeros((CLASS_NUM*BATCH_NUM_PER_CLASS,1,224,224), dtype=np.float32)
      class_cnt = 0

      for i in chosen_classes:
        # print ('class %s is chosen' % i)
        imgnames = os.listdir('FewShot/fss1000/%s/label' % classes_name[i])
        indexs = list(range(0,len(imgnames)))
        #print(len(indexs))
        chosen_index = random.sample(indexs, SAMPLE_NUM_PER_CLASS + BATCH_NUM_PER_CLASS)
        #print(len(chosen_index))
        j = 0
        for k in chosen_index:
            # process image
            image = cv2.imread('FewShot/fss1000/%s/image/%s' % (classes_name[i], imgnames[k].replace('.png', '.jpg')))
            image = cv2.resize(image,(224,224))
            if image is None:
                print ('FewShot/fss1000/%s/image/%s' % (classes_name[i], imgnames[k].replace('.png', '.jpg')))
                
            image = image[:,:,::-1] # bgr to rgb
            image = image / 255.0
            image = np.transpose(image, (2,0,1))
            # labels
            label = cv2.imread('FewShot/fss1000/%s/label/%s' % (classes_name[i], imgnames[k]))[:,:,0]
            label = cv2.resize(label,(224,224))
            label = label/255.0
            
            if j < SAMPLE_NUM_PER_CLASS:
                support_images[j] = image
                support_labels[j][0] = label
            else:
                query_images[j-SAMPLE_NUM_PER_CLASS] = image
                query_labels[j-SAMPLE_NUM_PER_CLASS][class_cnt] = label
            j += 1

        class_cnt += 1
      support_images_tensor = torch.from_numpy(support_images)
      support_labels_tensor = torch.from_numpy(support_labels)
      support_images_tensor = torch.cat((support_images_tensor,support_labels_tensor), dim=1)

      zeros_tensor = torch.from_numpy(zeros)
      query_images_tensor = torch.from_numpy(query_images)
      query_images_tensor = torch.cat((query_images_tensor,zeros_tensor), dim=1)
      query_labels_tensor = torch.from_numpy(query_labels)

      return support_images_tensor, support_labels_tensor, query_images_tensor, query_labels_tensor, chosen_classes
          





In [ ]:
samples, sample_labels, batches, batch_labels, chosen_classes = get_oneshot_batch()
print(samples.shape)
print(sample_labels.shape)
print(batches.shape)
print(batch_labels.shape)
print(chosen_classes)

torch.Size([5, 4, 224, 224])
torch.Size([5, 1, 224, 224])
torch.Size([5, 4, 224, 224])
torch.Size([5, 1, 224, 224])
[636]


In [ ]:
print(torch.max(batch_labels))

tensor(1.)


In [ ]:
def get_pascal_labels():
    """Load the mapping that associates pascal classes with label colors

    Returns:
        np.ndarray with dimensions (21, 3)
    """
    return np.asarray([[0,0,0], [128,0,0], [0,128,0], [128,128,0],
                      [0,0,128], [128,0,128], [0,128,128], [128,128,128],
                      [64,0,0], [192,0,0], [64,128,0], [192,128,0],
                      [64,0,128], [192,0,128], [64,128,128], [192,128,128],
                      [0, 64,0], [128, 64, 0], [0,192,0], [128,192,0],
                      [0,64,128]])


def encode_segmap(mask):
    """Encode segmentation label images as pascal classes

    Args:
        mask (np.ndarray): raw segmentation label image of dimension
          (M, N, 3), in which the Pascal classes are encoded as colours.

    Returns:
        (np.ndarray): class map with dimensions (M,N), where the value at
        a given location is the integer denoting the class index.
    """
    mask = mask.astype(int)
    label_mask = np.zeros((mask.shape[0], mask.shape[1]), dtype=np.int16)
    for ii, label in enumerate(get_pascal_labels()):
        label_mask[np.where(np.all(mask == label, axis=-1))[:2]] = ii
    label_mask = label_mask.astype(int)
    return label_mask


def decode_segmap(label_mask, plot=False):
    """Decode segmentation class labels into a color image

    Args:
        label_mask (np.ndarray): an (M,N) array of integer values denoting
          the class label at each spatial location.
        plot (bool, optional): whether to show the resulting color image
          in a figure.

    Returns:
        (np.ndarray, optional): the resulting decoded color image.
    """
    label_colours = get_pascal_labels()
    r = label_mask.copy()
    g = label_mask.copy()
    b = label_mask.copy()
    for ll in range(0, 21):
        r[label_mask == ll] = label_colours[ll, 0]
        g[label_mask == ll] = label_colours[ll, 1]
        b[label_mask == ll] = label_colours[ll, 2]
    rgb = np.zeros((label_mask.shape[0], label_mask.shape[1], 3))
    rgb[:, :, 0] = r #/ 255.0
    rgb[:, :, 1] = g #/ 255.0
    rgb[:, :, 2] = b #/ 255.0
    if plot:
        plt.imshow(rgb)
        plt.show()
    else:
        return rgb


In [ ]:
def evaluate_predictions(predictions, labels):
    pred = predictions.data.cpu().numpy()
    
    pred[pred <= 0.5] = 0
    pred[pred > 0.5] = 1
    testlabel = labels.numpy()[0][0].astype(bool)
    pred = pred.astype(bool)
    # Compute IOU
    overlap = testlabel * pred
    union = testlabel + pred
    iou = overlap.sum() / float(union.sum())
   
    return iou


In [ ]:
def main():

    
    print("Neural Networks")

    encoder = get_encoder('efficientnet-b4',in_channels=4,weights=None)
    decoder = UnetDecoder()
    head = SegmentationHead(in_channels=16,out_channels=1)


    initialize_decoder(decoder)
    initialize_head(head) 

    encoder.cuda(GPU)
    decoder.cuda(GPU)
    head.cuda(GPU) 



    encoder_optim = torch.optim.Adam(encoder.parameters(),lr=LEARNING_RATE)
    encoder_scheduler = StepLR(encoder_optim,step_size=EPISODE//10,gamma=0.5)

    decoder_optim = torch.optim.Adam(decoder.parameters(),lr=LEARNING_RATE)
    decoder_scheduler = StepLR(decoder_optim,step_size=EPISODE//10,gamma=0.5)

    head_optim = torch.optim.Adam(head.parameters(),lr=LEARNING_RATE)
    head_scheduler = StepLR(head_optim,step_size=EPISODE//10,gamma=0.5)


    if (args.finetune):

      if os.path.exists(ENCODER_MODEL):

          checkpoint1 = torch.load(ENCODER_MODEL)
          encoder.load_state_dict(checkpoint1['encoder_state_dict'])
          encoder_optim.load_state_dict(checkpoint1['optimizer_state_dict'])
          encoder_scheduler.load_state_dict(checkpoint1['scheduler_state_dict'])

       
          print("load encoder success")

      else:
          print('Can not load encoder: %s' % ENCODER_MODEL)
          print('starting from scratch')
          
      if os.path.exists(DECODER_MODEL):

          checkpoint2 = torch.load(DECODER_MODEL)
          decoder.load_state_dict(checkpoint2['decoder_state_dict'])
          decoder_optim.load_state_dict(checkpoint2['optimizer_state_dict'])
          decoder_scheduler.load_state_dict(checkpoint2['scheduler_state_dict'])

          print("load decoder success")

      else:
          print('Can not load decoder: %s' % DECODER_MODEL)
          print('starting from scratch')

      if os.path.exists(HEAD_MODEL):

          checkpoint3 = torch.load(HEAD_MODEL)
          head.load_state_dict(checkpoint3['head_state_dict'])
          head_optim.load_state_dict(checkpoint3['optimizer_state_dict'])
          head_scheduler.load_state_dict(checkpoint3['scheduler_state_dict'])

          print("load head success")

      else:
        print('Can not load head: %s' % HEAD_MODEL)
        print('starting from scratch')
    




    print("Training...")

    last_accuracy = 0.0
     
     
    for episode in range(args.start_episode, EPISODE):
        encoder_scheduler.step(episode)
        decoder_scheduler.step(episode)
        head_scheduler.step(episode)

        samples, sample_labels, batches, batch_labels, chosen_classes = get_oneshot_batch()


        # calculate features
        sample_features, _ = encoder(Variable(samples).cuda(GPU))  ## 5*384*7*7
        sample_features = sample_features.view(CLASS_NUM,SAMPLE_NUM_PER_CLASS,448,7,7)  ## 1*5*384*7*7
        sample_features = torch.sum(sample_features,1).squeeze(1) # 1*384*7*7
        batch_features, ft_list = encoder(Variable(batches).cuda(GPU))  ## 5*384*7*7

        # calculate relations
        sample_features_ext = sample_features.unsqueeze(0).repeat(BATCH_NUM_PER_CLASS*CLASS_NUM,1,1,1,1) ## 5*384*7*7
        batch_features_ext = batch_features.unsqueeze(0).repeat(CLASS_NUM,1,1,1,1)
        batch_features_ext = torch.transpose(batch_features_ext,0,1) ## 5*1*384*7*7

        relation_pairs = torch.cat((sample_features_ext,batch_features_ext),2).view(-1,896,7,7) ## 5*768*7*7
        out = decoder(relation_pairs,ft_list)
        output = head(out).view(-1,CLASS_NUM,224,224)

        bce = nn.BCELoss().cuda(GPU)
        loss = bce(output,Variable(batch_labels).cuda(GPU))


        # training

        encoder.zero_grad()
        decoder.zero_grad()
        head.zero_grad()

        loss.backward()


        torch.nn.utils.clip_grad_norm(encoder.parameters(),0.5)
        torch.nn.utils.clip_grad_norm(decoder.parameters(),0.5)
        torch.nn.utils.clip_grad_norm(head.parameters(),0.5)

        encoder_optim.step()
        decoder_optim.step()
        head_optim.step()

        if (episode+1)%10 == 0:
                
                print("episode: ",episode+1," loss: ",loss.cpu().data.numpy())
                

        if not os.path.exists(args.TrainResultPath):
            os.makedirs(args.TrainResultPath)
        if not os.path.exists(args.ModelSavePath):
            os.makedirs(args.ModelSavePath)

        # training result visualization
        if (episode+1)%args.ResultSaveFreq == 0:
          support_output = np.zeros((224*2, 224*SAMPLE_NUM_PER_CLASS, 3), dtype=np.uint8)
          query_output = np.zeros((224*3, 224*DISPLAY_QUERY, 3), dtype=np.uint8)
          chosen_query = random.sample(list(range(0,BATCH_NUM_PER_CLASS)), DISPLAY_QUERY)

          for i in range(CLASS_NUM):
              for j in range(SAMPLE_NUM_PER_CLASS):
                  supp_img = (np.transpose(samples.numpy()[j],(1,2,0))*255).astype(np.uint8)[:,:,:3][:,:,::-1]
                  support_output[0:224,j*224:(j+1)*224,:] = supp_img
                  supp_label = sample_labels.numpy()[j][0]
                  supp_label[supp_label!=0] = chosen_classes[i]
                  supp_label = decode_segmap(supp_label)
                  support_output[224:224*2, j*224:(j+1)*224,:] = supp_label

              for cnt, x in enumerate(chosen_query):
                  query_img = (np.transpose(batches.numpy()[x],(1,2,0))*255).astype(np.uint8)[:,:,:3][:,:,::-1]
                  query_output[0:224,cnt*224:(cnt+1)*224,:] = query_img
                  query_label = batch_labels.numpy()[x][0] #only apply to one-way setting
                  query_label[query_label!=0] = chosen_classes[i]
                  query_label = decode_segmap(query_label)
                  query_output[224:224*2, cnt*224:(cnt+1)*224,:] = query_label

                  query_pred = output.detach().cpu().numpy()[x][0]
                  query_pred = (query_pred*255).astype(np.uint8)
                  result = np.zeros((224,224,3), dtype=np.uint8)
                  result[:,:,0] = query_pred
                  result[:,:,1] = query_pred
                  result[:,:,2] = query_pred
                  query_output[224*2:224*3, cnt*224:(cnt+1)*224,:] = result
          extra = query_output.copy()
          for i in range(CLASS_NUM):
              for cnt, x in enumerate(chosen_query):
                  extra_label = batch_labels.numpy()[x][0]
                  extra_label[extra_label!=0] = 255
                  result1 = np.zeros((224,224,3), dtype=np.uint8)
                  result1[:,:,0] = extra_label
                  result1[:,:,1] = extra_label
                  result1[:,:,2] = extra_label
                  extra[224*2:224*3, cnt*224:(cnt+1)*224,:] = result1
          cv2.imwrite('%s/%s_query.png' % (args.TrainResultPath, episode), query_output)
          cv2.imwrite('%s/%s_show.png' % (args.TrainResultPath,episode), extra)
          cv2.imwrite('%s/%s_support.png' % (args.TrainResultPath, episode), support_output)
       


        if (episode+1) % args.ModelSaveFreq == 0:

            torch.save({'encoder_state_dict':encoder.state_dict(),
                        'optimizer_state_dict':encoder_optim.state_dict(),
                        'scheduler_state_dict':encoder_scheduler.state_dict()},str("./%s/efficientnet_encoder_" % args.ModelSavePath + str(episode) + '_' + str(CLASS_NUM) +"_way_" + str(SAMPLE_NUM_PER_CLASS) +"shot.pkl"))
            
            
            
            torch.save({'decoder_state_dict':decoder.state_dict(),
                        'optimizer_state_dict':decoder_optim.state_dict(),
                        'scheduler_state_dict':decoder_scheduler.state_dict()},str("./%s/decoder_network_" % args.ModelSavePath + str(episode) + '_' + str(CLASS_NUM) +"_way_" + str(SAMPLE_NUM_PER_CLASS) +"shot.pkl"))
            
            
            torch.save({'head_state_dict':head.state_dict(),
                        'optimizer_state_dict':head_optim.state_dict(),
                        'scheduler_state_dict':head_scheduler.state_dict()},str("./%s/segmentation_network_" % args.ModelSavePath + str(episode) + '_' + str(CLASS_NUM) +"_way_" + str(SAMPLE_NUM_PER_CLASS) +"shot.pkl"))
            
            print("save networks for episode:",episode)
            


In [ ]:
if __name__ == '__main__':
    main()

Neural Networks
Training from Scratch !!
Can not load encoder: 
starting from scratch
Can not load decoder: 
starting from scratch
Can not load head: 
starting from scratch
Training...


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:154: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://git

Streaming output truncated to the last 5000 lines.
episode:  52070  loss:  0.10759244
episode:  52080  loss:  0.08770678
episode:  52090  loss:  0.050978933
episode:  52100  loss:  0.038632482
episode:  52110  loss:  0.052049637
episode:  52120  loss:  0.031684354
episode:  52130  loss:  0.024447978
episode:  52140  loss:  0.054388378
episode:  52150  loss:  0.11113424
episode:  52160  loss:  0.04284343
episode:  52170  loss:  0.039474946
episode:  52180  loss:  0.10397655
episode:  52190  loss:  0.045678217
episode:  52200  loss:  0.040629197
episode:  52210  loss:  0.039488055
episode:  52220  loss:  0.028110584
episode:  52230  loss:  0.043516926
episode:  52240  loss:  0.08189462
episode:  52250  loss:  0.05207072
episode:  52260  loss:  0.0639707
episode:  52270  loss:  0.12963118
episode:  52280  loss:  0.020001614
episode:  52290  loss:  0.02833285
episode:  52300  loss:  0.12208413
episode:  52310  loss:  0.044812128
episode:  52320  loss:  0.03172877
episode:  52330  loss:  0.

KeyboardInterrupt: ignored

In [ ]:
!cp /content/models_newvgg_1shot/efficientnet_encoder_99999_1_way_5shot.pkl /content/drive/MyDrive/CGM

In [ ]:
while True:pass

KeyboardInterrupt: ignored